## Basic API Usage of KGWAS

KGWAS consists of two main class `KGWAS` and `KGWAS_Data`. `KGWAS` is the main class for the KGWAS model, and `KGWAS_Data` is the class for the data manipulation. In default, to ensure fast user experience, we provide a default fast mode of KGWAS, which uses Enformer embedding for variant feature and ESM embedding for gene features (instead of the baselineLD for variant and PoPS for gene since they are large files). For the fast mode, you do not need to download any data, the KGWAS API will automatically download the relevant files. This mode can be used to apply KGWAS to your own GWAS sumstats. 

In [1]:
import sys
sys.path.append('../')

from kgwas import KGWAS, KGWAS_Data
data = KGWAS_Data(data_path = './data/')
data.load_kg()

All required data files are present.
--loading KG---
--using enformer SNP embedding--
--using random go embedding--
--using ESM gene embedding--


Now, the data needed for training is downloaded from the server and the knowledge graph is loaded. Next, we load the GWAS file. Here, we are using an example GWAS file, which is also automatically downloaded from the server. But you can also use your own GWAS file. The GWAS file should be in the format of a pandas DataFrame with columns `CHR`/`#CHROM`, `SNP`, `P`, `N`. Note that at the moment, our knowledge graph is UKBioBank directly genotyped variant set so it will automatically takes the overlap with the KG. Current efforts are underway for improving the coverage of the KG.

In [2]:
data.load_external_gwas(example_file = True)
data.process_gwas_file()
data.prepare_split()

Loading example GWAS file...
Example file already exists locally.
Loading GWAS file from ./data/biochemistry_Creatinine_fastgwa_full_10000_1.fastGWA...
Using ldsc weight...
ldsc_weight mean:  0.9999999999999993


In [3]:
data.lr_uni

,#CHROM,ID,POS,A1,A2,N,AF1,BETA,SE,P,ld_score,w_ld_score,y
0,1,rs3131962,756604,A,G,9988,0.131007,-0.117134,0.246231,0.634282,72.862240,4.474788,0.226298
1,1,rs12562034,768448,A,G,9978,0.104981,-0.064894,0.273746,0.812611,34.749233,1.877341,0.056197
2,1,rs4040617,779322,G,A,9975,0.129123,-0.001462,0.247254,0.995281,72.271390,4.208873,0.000035
3,1,rs79373928,801536,G,T,9994,0.014659,0.081544,0.688261,0.905688,16.740126,1.949177,0.014037
4,1,rs11240779,808631,G,A,9919,0.226737,-0.184268,0.198982,0.354418,50.215000,2.825456,0.857575
...,...,...,...,...,...,...,...,...,...,...,...,...,...
542753,22,rs73174435,51174939,T,C,9979,0.056118,-0.158762,0.362390,0.661316,21.981667,1.363001,0.191929
542754,22,rs3810648,51175626,G,A,9931,0.058856,0.272493,0.352508,0.439515,34.619377,1.804193,0.597548
542755,22,rs5771002,51183255,A,G,9840,0.333638,0.116325,0.175675,0.507869,16.231083,1.273770,0.438456
542756,22,rs3865764,51185848,G,A,9974,0.051133,-0.026670,0.376132,0.943472,18.649513,1.010000,0.005028


Next, we are ready to train the model! Here we are using epoch = 1 for the demo purpose, but in reality, you should use a higher number of epochs for better performance.

In [4]:
run = KGWAS(data, device = 'cuda:9', exp_name = 'test')
run.initialize_model()
run.train(epoch = 1)

Creating data loader...
Start Training...
100%|██████████| 50/50 [00:58<00:00,  1.17s/it]
Epoch 1: Validation MSE: 2.1730 Validation Pearson: 0.0096. 
Saving models to ./data//model/test
100%|██████████| 1061/1061 [05:40<00:00,  3.11it/s]


KGWAS prediction and p-values saved to ./data//model_pred/new_experiments/test_pred.csv


The output of the model is saved to `/model_pred/new_experiments/{exp_name}_pred.csv`. You can also load it via `run.kgwas_res`. The model is also saved to `/model/{exp_name}`.

In [5]:
run.kgwas_res

,#CHROM,ID,POS,A1,A2,N,AF1,BETA,SE,P,ld_score,w_ld_score,y,pred,P_weighted,KGWAS_P
0,1,rs3131962,756604,A,G,9988,0.131007,-0.117134,0.246231,0.634282,72.862240,4.474788,0.226298,1.082365,0.234167,0.346428
1,1,rs12562034,768448,A,G,9978,0.104981,-0.064894,0.273746,0.812611,34.749233,1.877341,0.056197,1.087724,0.382894,0.566456
2,1,rs4040617,779322,G,A,9975,0.129123,-0.001462,0.247254,0.995281,72.271390,4.208873,0.000035,1.058530,0.995281,1
3,1,rs79373928,801536,G,T,9994,0.014659,0.081544,0.688261,0.905688,16.740126,1.949177,0.014037,1.105125,0.225107,0.333025
4,1,rs11240779,808631,G,A,9919,0.226737,-0.184268,0.198982,0.354418,50.215000,2.825456,0.857575,1.081468,0.041646,0.061612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542753,22,rs73174435,51174939,T,C,9979,0.056118,-0.158762,0.362390,0.661316,21.981667,1.363001,0.191929,1.008835,0.233609,0.345602
542754,22,rs3810648,51175626,G,A,9931,0.058856,0.272493,0.352508,0.439515,34.619377,1.804193,0.597548,1.034187,0.439515,0.650221
542755,22,rs5771002,51183255,A,G,9840,0.333638,0.116325,0.175675,0.507869,16.231083,1.273770,0.438456,1.093221,0.449038,0.66431
542756,22,rs3865764,51185848,G,A,9974,0.051133,-0.026670,0.376132,0.943472,18.649513,1.010000,0.005028,0.987747,0.943472,1


If needed, you can load the pre-trained model via `run.load_pretrained()`.

In [ ]:
run.load_pretrained('./data/model/test')

If you want to (1) use the full mode of KGWAS (i.e. larger node embeddings) or (2) access the null/causal simulations or (3) access the 21 subsampled GWAS sumstats across various sample sizes or (4) analyze the KGWAS sumstats for subsampled data or (5) analyze the KGWAS sumstats for all UKBB ICD10 diseases, please use [this link](https://drive.google.com/file/d/14UcHzPRIbdMmnLPZCHx_4G-gz2pipeg9/view?usp=sharing). Note that this file is large (around 45GB) and may take a while to download. After unzipping it, you can use that directory as the data directory for the KGWAS API.

In [2]:
from kgwas import KGWAS, KGWAS_Data
data = KGWAS_Data(data_path = '/dfs/project/datasets/20220524-ukbiobank/data/kgwas_data/')

All required data files are present.


Now that you can use various variant, gene, and program embeddings. For example, for the result in the paper, we use the baselineLD for variant and PoPS for gene.

In [3]:
data.load_kg(snp_init_emb = 'baselineLD', 
             go_init_emb = 'random',
             gene_init_emb = 'pops')

--loading KG---
--using baselineLD SNP embedding--
--using random go embedding--
--using PoPs expression+PPI+pathways gene embedding--


There are many alternative embeddings as well. 
- For variant: `enformer` (default), `baselineLD`, `SLDSC`, `cadd`, `kg`, `random`
- For gene: `esm` (default), `pops_expression`, `pops`, `kg`, `random`
- For program/go: `random` (default), `biogpt`, `kg`

In additional to more embeddings, the full data folder contains summary statistics used in each analysis in the paper. For example, for the simulations, you can load it via:

In [ ]:
data.load_simulation_gwas('causal', seed = 1) # seed can range from 1-500
data.lr_uni

All required data files are present.
Using simulation data....


,#CHROM,ID,POS,A1,A2,N,AF1,BETA,SE,P
0,1,rs3131962,756604,A,G,4993,0.129882,14.559400,17.1871,0.396933
1,1,rs12562034,768448,A,G,4994,0.103124,-15.034400,19.0234,0.429345
2,1,rs4040617,779322,G,A,4979,0.127435,15.537200,17.3933,0.371704
3,1,rs79373928,801536,G,T,4996,0.015012,16.142600,47.7752,0.735448
4,1,rs11240779,808631,G,A,4961,0.222233,0.859838,13.9158,0.950731
...,...,...,...,...,...,...,...,...,...,...
542753,22,rs73174435,51174939,T,C,4991,0.057103,53.082400,24.8130,0.032412
542754,22,rs3810648,51175626,G,A,4959,0.066243,17.689800,23.2562,0.446867
542755,22,rs5771002,51183255,A,G,4937,0.334414,-12.170400,12.3314,0.323670
542756,22,rs3865764,51185848,G,A,4984,0.050662,-43.871900,26.3007,0.095299


Similarly for null simulations, you can load it via:

In [2]:
data.load_simulation_gwas('null', seed = 1)# seed can range from 1-500
data.lr_uni

Using simulation data....


,#CHROM,ID,POS,A1,A2,N,AF1,BETA,SE,P
0,1,rs3131962,756604,A,G,4993,0.129882,-2.960260,7.66276,0.699261
1,1,rs12562034,768448,A,G,4994,0.103124,-19.335700,8.47710,0.022552
2,1,rs4040617,779322,G,A,4979,0.127435,-3.287600,7.75475,0.671605
3,1,rs79373928,801536,G,T,4996,0.015012,-12.530000,21.29860,0.556329
4,1,rs11240779,808631,G,A,4961,0.222233,-8.564830,6.20273,0.167335
...,...,...,...,...,...,...,...,...,...,...
542753,22,rs73174435,51174939,T,C,4991,0.057103,-24.859400,11.06160,0.024617
542754,22,rs3810648,51175626,G,A,4959,0.066243,-0.725793,10.36870,0.944195
542755,22,rs5771002,51183255,A,G,4937,0.334414,-5.555300,5.49753,0.312251
542756,22,rs3865764,51185848,G,A,4984,0.050662,12.588200,11.72730,0.283085


Now, for the subsampling analysis, you can load any trait out of the 21 subsampled traits in various sample sizes across 5 replicates. The phenotype list can be accessed via:

In [3]:
data.get_pheno_list()['21_indep_traits']

['body_BALDING1',
 'disease_ALLERGY_ECZEMA_DIAGNOSED',
 'disease_HYPOTHYROIDISM_SELF_REP',
 'pigment_SUNBURN',
 '21001',
 '50',
 '30080',
 '30070',
 '30010',
 '30000',
 'biochemistry_AlkalinePhosphatase',
 'biochemistry_AspartateAminotransferase',
 'biochemistry_Cholesterol',
 'biochemistry_Creatinine',
 'biochemistry_IGF1',
 'biochemistry_Phosphate',
 'biochemistry_Testosterone_Male',
 'biochemistry_TotalBilirubin',
 'biochemistry_TotalProtein',
 'biochemistry_VitaminD',
 'bmd_HEEL_TSCOREz']

Usually each trait has the following sample sizes available: 1000, 2500, 5000, 7500, 10000, 50000, 100000, 200000. For example, to load body_BALDING1 at sample size 1000 at replicate 1, you can use:

In [4]:
data.load_gwas_subsample(pheno = 'body_BALDING1', sample_size = 1000, seed = 1)
data.lr_uni

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE,SNP,A2,N
0,1,756604,rs3131962,G,A,A,Y,ADD,999,1.241130,0.209870,1.029320,0.303330,.,rs3131962,G,999
1,1,768448,rs12562034,G,A,A,Y,ADD,996,0.433894,0.285912,-2.920330,0.003497,.,rs12562034,G,996
2,1,779322,rs4040617,A,G,G,Y,ADD,996,1.178310,0.211892,0.774379,0.438707,.,rs4040617,A,996
3,1,801536,rs79373928,T,G,G,Y,ADD,998,0.989852,0.479159,-0.021286,0.983018,.,rs79373928,T,998
4,1,808631,rs11240779,A,G,G,Y,ADD,994,0.880382,0.173114,-0.735930,0.461773,.,rs11240779,A,994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542753,22,51174939,rs73174435,C,T,T,Y,ADD,999,0.642727,0.362564,-1.219190,0.222772,.,rs73174435,C,999
542754,22,51175626,rs3810648,A,G,G,Y,ADD,996,0.752885,0.286799,-0.989690,0.322326,.,rs3810648,A,996
542755,22,51183255,rs5771002,G,A,A,Y,ADD,981,0.792577,0.150356,-1.546100,0.122080,.,rs5771002,G,981
542756,22,51185848,rs3865764,A,G,G,Y,ADD,996,1.004930,0.386700,0.012715,0.989855,.,rs3865764,A,996


You can also load the full cohort GWAS for these 21 traits via:

In [ ]:
data.load_full_gwas(pheno = 'body_BALDING1')
data.lr_uni

,#CHROM,ID,POS,A1,A2,N,AF1,BETA,SE,P
0,1,rs3131962,756604,A,G,407023,0.129655,0.000286,0.001048,0.784760
1,1,rs12562034,768448,A,G,407057,0.104966,-0.001491,0.001147,0.193592
2,1,rs4040617,779322,G,A,406623,0.127520,0.000108,0.001056,0.918404
3,1,rs79373928,801536,G,T,407517,0.014884,0.004382,0.002904,0.131349
4,1,rs11240779,808631,G,A,404493,0.224886,-0.001155,0.000846,0.172345
...,...,...,...,...,...,...,...,...,...,...
542753,22,rs73174435,51174939,T,C,407201,0.053846,-0.001980,0.001559,0.203959
542754,22,rs3810648,51175626,G,A,404901,0.060979,0.001922,0.001474,0.192116
542755,22,rs5771002,51183255,A,G,401398,0.333603,-0.000165,0.000751,0.826494
542756,22,rs3865764,51185848,G,A,406611,0.050601,-0.001311,0.001605,0.413994


This is the basic KGWAS interface! Check out the other notebooks for other capabilities of KGWAS!